# Basic Proxy Signature Protocol for Partial Delegation

## Greatest Common Divisor

We need this function in the signature scheme as a utility function.

In [23]:
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("wrong parameter input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r

    return a

The usage of gcd function is tested with some examples as follows:

In [24]:
a = 31
b = 23
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 16
b = 9
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 468
b = 249
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

gcd( 31 , 23 )= 1
gcd( 16 , 9 )= 1
gcd( 468 , 249 )= 3


# Protocol

## Setup: Key pair generation of original signer

public parameter - modulus: prime $p$

public parameter - generator for $Z^*_p$: $g$

public key: v

secret key: s

__In this test implementation, only input needs to be given by the user is the prime modulus $p$.__

The key pair generation process can be modified as follows to allow the use of a large prime number ***p*** and efficiently find a generator ***g*** for a multiplicative group of integers modulo ***p***.

Using Miller Rabin Primality test, a given number ***n*** can be efficiently check for primality.

In [25]:
import random

#Miller Rabin Primality Test
def is_prime(n, k=5):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        r += 1

    #check if a base a indicates n is composite.
    def check_composite(a):
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            return False
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                return False
        return True

    for _ in range(k):
        a = random.randint(2, n - 2)
        if check_composite(a):
            return False
    return True

# Generates a large prime number with a specified number of bits.
def generate_large_prime(bits=32):
    while True:
        p = random.getrandbits(bits) | 1  # Ensures p is odd
        if is_prime(p):
            return p

# Finds a generator (primitive root) for the multiplicative group of integers modulo p.
def find_generator(p):
    phi = p - 1
    factors = {i for i in range(2, int(phi**0.5) + 1) if phi % i == 0}
    factors.update({phi // f for f in factors})

    for g in range(2, p):
        if all(pow(g, phi // f, p) != 1 for f in factors):
            return g
    return None

p = generate_large_prime()
g = find_generator(p)

if g is None:
    print("No generator found for the group.")
else:
    s = random.randint(1, p - 2)
    v = pow(g, s, p)

    print('*public parameters*')
    print('prime modulus (p):', p)
    print('generator     (g):', g)
    print('*key pair of original signer*')
    print('public key    (v):', v)
    print('secret key    (s):', s)


*public parameters*
prime modulus (p): 1410194329
generator     (g): 13
*key pair of original signer*
public key    (v): 875101051
secret key    (s): 897451663


## Step 1: Proxy generation

Proxy: ($\sigma$, K)

The values of ***k*** and  ***sigma*** can be efficiently generated as follows:

In [26]:
k = random.randint(1,p-2) # random number, Z_{p-1}/{0}
print('(just any) random number k:',k)

print('*proxy value pair for proxy signer*')

bigk = pow(g,k,p)
print('K    :',bigk)

sigma = (s + k*bigk)%(p-1)
print('sigma:',sigma)

(just any) random number k: 185672345
*proxy value pair for proxy signer*
K    : 380483764
sigma: 1368343731


## Step 2: Proxy delivery

The proxy $(\sigma,K)$, must be given by original signer to proxy signer securely.

For example, using Diffie-Hellman to create a secure tunnel, this can be done.

## Step 3: Proxy verification

The verification process can be optmized in an efficient way by seperately checking whether the 2 interpretatins are the same.

In [27]:
lhs = pow(g,sigma,p)

rhs = (v*pow(bigk,bigk,p))%p

print('LHS:',lhs)
print('RHS:',rhs)

if(lhs==rhs):
    print('Proxy Verification: Passed')
else:
    print('Proxy Verification: Failed')

LHS: 1379032947
RHS: 1379032947
Proxy Verification: Passed


## Step 4: Signing by the proxy signer

We are going to use __ElGamal signature scheme__.

The __ElGamal signing__ protocol to sign message $m$ where the secret key of the signer is $x$:
1. Choose an integer $k$ randomly from $\{ 2 \ldots p−2 \}$  with $k$ relatively prime to $p−1$.
2. Compute $r \equiv g^{k} \pmod p$.
3. Compute $s \equiv (H(m)-xr)k^{-1} \pmod {p-1}$.
If s = 0, then start again with a different random k.

The signature is $(r,s)$.

When generating the proxy signature, use $\sigma$ instead of the secret key $x$.

In [28]:
import hashlib
# m = random.randint(1,p-1) # message for signing. Consider this as H(m) in 1 ... p-1

# read the message from the file
#Req 3: Allow for an arbitrary size file to be read from the disk as input message for generating a proxy signature
def read_message_from_file(filename):
    with open(filename, 'r') as file:
        message = file.read()
        print(message)
    return int(hashlib.sha256(message.encode()).hexdigest(), 16)

m = read_message_from_file('message.txt')
print('message (m):',m)

k = 0

#Trying out 1000 random values for k to find one that is relatively prime to p-1
for i in range (1000): # this number 1000 is just a random value for the number of tries
    k = random.randint(2,p-2) # random number k is in {2 ... (p-2)} and gcd(k,(p-1))=1
    #print('testing',i,'th random number k:',k)
    d = gcd(p-1,k)
    if(d==1):
        break

print('(a special) random number k:',k,'(found after',i,'tries. This is relatively prime to p-1 =',p-1,')')

r = pow(g,k,p)
print('r :',r)

s1 = m - (sigma*r) # in place of s, use sigma, theproxy secret
print('s1:',s1)

Ado mama Anjula
message (m): 92936444892425296126897496249657084158512872626459328642006554000005634312468
(a special) random number k: 735528551 (found after 0 tries. This is relatively prime to p-1 = 1410194328 )
r : 661956901
s1: 92936444892425296126897496249657084158512872626459328642005648215429958774837


In [29]:
# Applies the Extended Euclidean Algorithm to determine the greatest common divisor of a and b, and the coefficients for Bézout's identity.
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x_prev, y_prev = extended_gcd(b % a, a)
    x_current = y_prev - (b // a) * x_prev
    y_current = x_prev
    return gcd, x_current, y_current

# Calculate the modular inverse of k modulo q using the Extended Euclidean Algorithm.
def modular_inverse(k, q):
    gcd, x, _ = extended_gcd(k, q)
    if gcd != 1:
        raise ValueError(f"Modular inverse doesn't exist for k = {k} under modulo q = {q}")
    return x % q

# Assume q = p-1 and k are already defined from previous steps
q = p - 1

try:
    invk = modular_inverse(k, q)
    print('Modulo inverse of k mod q:', invk, '(for', k, 'mod', q, ')')
except ValueError as e:
    print(e)


Modulo inverse of k mod q: 500922983 (for 735528551 mod 1410194328 )


In [30]:
# Calculate the value of the signature
s = (s1*invk)%(p-1)
print('s:',s)
print('The proxy signed message is (m,(r,s),K):(',m,'(',r,',',s,')',bigk,')')

# Write the signature components to a file.
# Req 4: Allow for the proxy signature to be written to the disk
def write_signature_to_file(filename, signature):
    with open(filename, 'w') as file:
        file.write(f"{signature[0]}\n{signature[1]}\n{signature[2]}")

write_signature_to_file('signature.txt', (r, s, bigk))
print('Signature written to signature.txt')

s: 476482739
The proxy signed message is (m,(r,s),K):( 92936444892425296126897496249657084158512872626459328642006554000005634312468 ( 661956901 , 476482739 ) 380483764 )
Signature written to signature.txt


## Step 5: Verification of the proxy signature

The new public key $v' \equiv vK^K \pmod p$

The __ElGamal signature verification__ protocol to verify a signed message $m$ with signature $(r,s)$ where the public key of the signer is $y$:
1. Verify that $0 < r < p$ and $0 < s < p − 1$.
2. Verify that $g^{H(m)} \equiv y^r r^s \pmod p$.

Req 5 is implemented below: Allow for a message and a proxy signature read from the disk to be verified

The process of verification of proxy signature can be optimized fr efficiency as follows:

In [31]:
# Read the signature form the file
def read_siganture(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        return int(lines[0]), int(lines[1]), int(lines[2])

r, s, bigk = read_siganture('signature.txt')

print('r =',r,', s =',s,', p =',p)
check1 = False
check2 = False

# Verification Check 1
if (0<r) & (r<p):
    if (0<s) & (s<(p-1)):
        check1 = True
        print('Verification Check 1: Passed')
    else:
        print('Verification Check 1: Failed')
else:
    print('Verification Check 1: Failed')

r = 661956901 , s = 476482739 , p = 1410194329
Verification Check 1: Passed


In [32]:
# Verification check 2
y = (v*pow(bigk,bigk,p))%p # this is the proxy public key y = VK^K mod p

lhs = pow(g,m,p)

rhs = (pow(y,r,p)*pow(r,s,p))%p

print('LHS:',lhs)
print('RHS:',rhs)

if(lhs==rhs):
    check2 = True
    print('Verification Check 2: Passed')
else:
    print('Verification Check 2: Failed')

LHS: 12462952
RHS: 12462952
Verification Check 2: Passed


Final verification process:

In [33]:
if(check1 & check2):
    print('Proxy signature verification: Passed')
else:
    print('Proxy signature verification: Failed')

Proxy signature verification: Passed
